In [ ]:
threshold_ss = 0.65
threshold_maintain = 0.85
threshold_non = 0.2
threshold_judge = 0


def get_table1_deprecated(data_csv_train, data_csv_test, stat_csv_23):
    data_new_diff_count_total = pd.read_csv(stat_csv_23)

    table1_columns = data_new_diff_count_total['steer_dim'].unique()
    table1_columns = table1_columns[~np.isnan(table1_columns)]
    value_dims = data_new_diff_count_total.columns[1:]
    table1 = pd.DataFrame(columns=table1_columns, index=value_dims)


    players_list_train = data_csv_train['player_name'].unique()
    #players_list_train = players_list_local[~pd.isnull(players_list_local)]

    players_list_test = data_csv_test['player_name'].unique()
    players_list_test = players_list_test[~pd.isnull(players_list_test)]

    standard_data = data_csv_test[data_csv_test['steer_dim'].isnull()]

    for steer_dim in table1_columns:
        assert not np.isnan(steer_dim)
        #print(steer_dim)

        steer_dim_row = data_new_diff_count_total[data_new_diff_count_total['steer_dim'] == steer_dim]
        stimulated_dims = []
        suppressed_dims = []
        maintained_dims = []
        non_suppressed_dims = []
        non_stimulated_dims = []
        uncontrolled_dims = []

        for column in value_dims:
            assert column != 'steer_dim'
            #split cell by /
            counts = steer_dim_row[column].values[0].split('/')   
            if int(counts[0]) / len(players_list_train) > threshold_ss:
                stimulated_dims.append(column)
            elif int(counts[1]) / len(players_list_train) > threshold_ss:
                suppressed_dims.append(column)
            elif int(counts[2]) / len(players_list_train) > threshold_maintain:
                maintained_dims.append(column)
            elif int(counts[1]) / len(players_list_train) < threshold_non:
                non_suppressed_dims.append(column)
            elif int(counts[0]) / len(players_list_train) < threshold_non:
                non_stimulated_dims.append(column)
            else:
                uncontrolled_dims.append(column)

        steer_dim_data = data_csv_test[data_csv_test['steer_dim'] == steer_dim]
        for value_dim in stimulated_dims:
            count_correct_steer = 0
            for player_name in players_list_test:
                steered_player_data = steer_dim_data[steer_dim_data['player_name'] == player_name][value_dim].values[0]
                standard_player_data = standard_data[standard_data['player_name'] == player_name][value_dim].values[0]
                if steered_player_data - standard_player_data > threshold_judge:
                    count_correct_steer += 1
            #print(value_dim, 'SITMULATE', count_correct_steer / len(players_list_test), count_correct_steer)
            #edit the table
            table1.loc[value_dim, steer_dim] = 'STIMULATE,' + str(count_correct_steer / len(players_list_test))

        for value_dim in suppressed_dims:
            count_correct_steer = 0
            for player_name in players_list_test:
                steered_player_data = steer_dim_data[steer_dim_data['player_name'] == player_name][value_dim].values[0]
                standard_player_data = standard_data[standard_data['player_name'] == player_name][value_dim].values[0]
                if -(steered_player_data - standard_player_data) > threshold_judge:
                    count_correct_steer += 1
            #print(value_dim, 'SUPPRESS', count_correct_steer / len(players_list_test), count_correct_steer)
            table1.loc[value_dim, steer_dim] = 'SUPPRESS,' + str(count_correct_steer / len(players_list_test))
            
        for value_dim in non_suppressed_dims:
            count_correct_steer = 0
            for player_name in players_list_test:
                steered_player_data = steer_dim_data[steer_dim_data['player_name'] == player_name][value_dim].values[0]
                standard_player_data = standard_data[standard_data['player_name'] == player_name][value_dim].values[0]
                if steered_player_data - standard_player_data >= -threshold_judge:
                    count_correct_steer += 1
            #print(value_dim, 'NON_SUPPRESS', count_correct_steer / len(players_list_test), count_correct_steer)
            table1.loc[value_dim, steer_dim] = 'NON_SUPPRESS,' + str(count_correct_steer / len(players_list_test))

        for value_dim in non_stimulated_dims:
            count_correct_steer = 0
            for player_name in players_list_test:
                steered_player_data = steer_dim_data[steer_dim_data['player_name'] == player_name][value_dim].values[0]
                standard_player_data = standard_data[standard_data['player_name'] == player_name][value_dim].values[0]
                if steered_player_data - standard_player_data <= threshold_judge:
                    count_correct_steer += 1
            #print(value_dim, 'NON_STIMULATE', count_correct_steer / len(players_list_test), count_correct_steer)   
            table1.loc[value_dim, steer_dim] = 'NON_STIMULATE,' + str(count_correct_steer / len(players_list_test))
        
        for value_dim in maintained_dims:
            count_correct_steer = 0
            for player_name in players_list_test:
                steered_player_data = steer_dim_data[steer_dim_data['player_name'] == player_name][value_dim].values[0]
                standard_player_data = standard_data[standard_data['player_name'] == player_name][value_dim].values[0]
                if abs(steered_player_data - standard_player_data) <= threshold_judge:
                    count_correct_steer += 1
            #print(value_dim, 'MAINTAIN', count_correct_steer / len(players_list_test), count_correct_steer)
            table1.loc[value_dim, steer_dim] = 'MAINTAIN,' + str(count_correct_steer / len(players_list_test))
    return table1


table1_gemma = get_table1_deprecated(data_csv_gemma_train, data_csv_gemma_test, 'value_dims_rsd_gemma/23_stat.csv')

In [ ]:
threshold_ss = 0.65
threshold_maintain = 0.85
threshold_non = 0.2
threshold_judge = 0

def get_table1_new(data_csv_train, data_csv_test, stat_csv_23):
    data_new_diff_count_total = pd.read_csv(stat_csv_23)

    table1_rows = data_new_diff_count_total['steer_dim'].unique()
    table1_rows = table1_rows[~np.isnan(table1_rows)]
    #table1_columns = data_new_diff_count_total.columns[1:]
    table1_columns = data_new_diff_count_total.columns[data_new_diff_count_total.columns != 'steer_dim']
    table1 = pd.DataFrame(columns=table1_columns, index=table1_rows)
    
    #players_list_train = data_csv_train['player_name'].unique()
    #players_list_train = players_list_local[~pd.isnull(players_list_local)]

    players_list_test = data_csv_test['player_name'].unique()
    players_list_test = players_list_test[~pd.isnull(players_list_test)]

    standard_data = data_csv_test[data_csv_test['steer_dim'].isnull()]

    for steer_dim in table1_rows:
        assert not np.isnan(steer_dim)
        #print(steer_dim)

        steer_dim_row = data_new_diff_count_total[data_new_diff_count_total['steer_dim'] == steer_dim]
        steer_dim_data = data_csv_test[data_csv_test['steer_dim'] == steer_dim]

        for column in table1_columns:
            assert column != 'steer_dim'
            #split cell by /
            counts = steer_dim_row[column].values[0].split('/')   

            simu = int(counts[0])
            supp = int(counts[1])
            main = int(counts[2])
            
            simu_test = 0
            supp_test = 0
            main_test = 0
            
            for player_name in players_list_test:
                steered_player_data = steer_dim_data[steer_dim_data['player_name'] == player_name][column].values[0]
                standard_player_data = standard_data[standard_data['player_name'] == player_name][column].values[0]
                if steered_player_data - standard_player_data > threshold_judge:
                    simu_test += 1
                elif -(steered_player_data - standard_player_data) > threshold_judge:
                    supp_test += 1
                elif abs(steered_player_data - standard_player_data) <= threshold_judge:
                    main_test += 1
                else:
                    raise ValueError('Invalid answer')
            table1.loc[steer_dim, column] = str(simu) + '/' + str(supp) + '/' + str(main) + '/' + str(simu_test) + '/' + str(supp_test) + '/' + str(main_test)
    return table1

table1_gemma = get_table1_new(data_csv_gemma_train, data_csv_gemma_test, 'value_dims_rsd_gemma/23_stat.csv')
table1_gemma.to_csv('table1_gemma.csv')

In [ ]:
def get_latex_table1_deprecated(table1, table1_name):
    latex_code = '\\begin{table*}[ht]\n\\caption{Value steering using SAE features for the Gemma-2B-IT model. Expected stimulated values are highlighted in red, along with their actual success rate during testing. Expected suppressed values are marked in Purple. Maintained values are shown in gray. Light red indicates values that are expected to be at least not suppressed, while light purple represents values that are expected to be at least not stimulated. Blank cells correspond to uncontrollable values. The bottom of the table indicates the count of each of the six expected categories and their average success rates.}\n\\label{table: sae-steering-gemma}\n\\begin{center}\n\\scalebox{0.5}{'
    #latex_code = '\\begin{table}[ht]\n\\caption{Value steering using SAE features for the Llama3-8B-IT model.}\n\\label{table: sae-steering-llama}\n\\begin{center}\n'

    latex_code += '\\begin{tabular}{c@{\\hspace{2pt}}' + 'c@{\\hspace{2pt}}' * (len(table1.columns) - 1) + 'c' + '}\n\\toprule\n'
    #transfer table1.columns to a list of str


    steering_features = list(map(str, map(int, table1.columns)))
    latex_code += 'Value & ' + ' & '.join(['\\bf ' + tc for tc in steering_features]) + ' \\\\\n\\hline\n'
    #

    stimulated_dim_avg_success = {sf: [] for sf in steering_features}
    stimulhalf_dim_avg_success = {sf: [] for sf in steering_features}
    suppressed_dim_avg_success = {sf: [] for sf in steering_features}
    supprehalf_dim_avg_success = {sf: [] for sf in steering_features}
    maintained_dim_avg_success = {sf: [] for sf in steering_features}


    uncontroll_dims = {sf: 0 for sf in steering_features}
    stimulated_dims = {sf: 0 for sf in steering_features} 
    suppressed_dims = {sf: 0 for sf in steering_features}
    stimulhalf_dims = {sf: 0 for sf in steering_features}
    supprehalf_dims = {sf: 0 for sf in steering_features}
    maintained_dims = {sf: 0 for sf in steering_features}


    for index, row in table1.iterrows():
        #if value's name (index) is too long, make its font smaller, all value names should be available in 3pt
        if len(index) > 20:
            latex_code += '\\tiny ' + index + ' & '
        else:
            latex_code += '\\small ' + index + ' & '

        for value, sf in zip(row, steering_features):
            if type(value) == str:
                print(value)
                value = value.split(',')
                
                if value[0] == 'STIMULATE':
                    stimulated_dim_avg_success[sf].append(float(value[1]))
                    stimulated_dims[sf] += 1
                    #latex_code += '\\textcolor{red}{\\textbf{$\\uparrow$}}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    latex_code += '\\colorbox{red!50}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    #latex_code += f"{float(value[1]):.2f}" + ' & '
                elif value[0] == 'NON_SUPPRESS':
                    stimulhalf_dim_avg_success[sf].append(float(value[1]))
                    stimulhalf_dims[sf] += 1
                    #latex_code += '\\textcolor{magenta}{\\textbf{$\\nearrow$}}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    latex_code += '\\colorbox{red!20}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    #latex_code += f"{float(value[1]):.2f}" + ' & '
                elif value[0] == 'SUPPRESS':
                    suppressed_dim_avg_success[sf].append(float(value[1]))
                    suppressed_dims[sf] += 1
                    #latex_code += '\\textcolor{blue}{\\textbf{$\\downarrow$}}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    latex_code += '\\colorbox{blue!50}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    #latex_code += f"{float(value[1]):.2f}" + ' & '
                elif value[0] == 'NON_STIMULATE':
                    supprehalf_dim_avg_success[sf].append(float(value[1]))
                    supprehalf_dims[sf] += 1
                    #latex_code += '\\textcolor{cyan}{\\textbf{$\\searrow$}}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    latex_code += '\\colorbox{blue!20}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    #latex_code += f"{float(value[1]):.2f}" + ' & '
                elif value[0] == 'MAINTAIN':
                    maintained_dim_avg_success[sf].append(float(value[1]))
                    maintained_dims[sf] += 1
                    #latex_code += '\\textcolor{purple}{\\textbf{-}}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    latex_code += '\\colorbox{gray!20}' + ' ' + f"{float(value[1]):.2f}" + ' & '
                    #latex_code += f"{float(value[1]):.2f}" + ' & '
                else:
                    raise ValueError('Invalid value')
            else:
                assert np.isnan(value)
                uncontroll_dims[sf] += 1
                #latex_code += '\\textcolor{gray}{-} & '
                latex_code += f"-" + ' & '
                #latex_code += '- & '
        latex_code = latex_code[:-2] + ' \\\\\n'
    latex_code = latex_code + ' \\midrule\n'

    for sf in steering_features:
        stimulated_dim_avg_success[sf] = np.mean(stimulated_dim_avg_success[sf])
        stimulhalf_dim_avg_success[sf] = np.mean(stimulhalf_dim_avg_success[sf])
        suppressed_dim_avg_success[sf] = np.mean(suppressed_dim_avg_success[sf])
        supprehalf_dim_avg_success[sf] = np.mean(supprehalf_dim_avg_success[sf])
        maintained_dim_avg_success[sf] = np.mean(maintained_dim_avg_success[sf])
        
    latex_code += '\\colorbox{red!50} STIMULATE & '
    for sf in steering_features:
        cellcontent = round(stimulated_dim_avg_success[sf],3)
        latex_code += '\\textbf{' + str(stimulated_dims[sf]) + f'({cellcontent})' +'} & '
    latex_code = latex_code[:-2] + ' \\\\\n'

    latex_code += '\\colorbox{blue!50} SUPPRESSED & '
    for sf in steering_features:
        cellcontent = round(suppressed_dim_avg_success[sf],3)
        latex_code += '\\textbf{' + str(suppressed_dims[sf]) + f'({cellcontent})' +'} & '
    latex_code = latex_code[:-2] + ' \\\\\n'

    latex_code += '\\colorbox{red!20} NON-SUPPRESSED & '
    for sf in steering_features:
        cellcontent = round(stimulhalf_dim_avg_success[sf],3)
        latex_code += '\\textbf{' + str(stimulhalf_dims[sf]) + f'({cellcontent})' +'} & '
    latex_code = latex_code[:-2] + ' \\\\\n'

    latex_code +='\\colorbox{blue!20} NON-STIMULATED & '
    for sf in steering_features:
        cellcontent = round(supprehalf_dim_avg_success[sf],3)
        latex_code += '\\textbf{' + str(supprehalf_dims[sf]) + f'({cellcontent})' +'} & '
    latex_code = latex_code[:-2] + ' \\\\\n'

    latex_code += '\\colorbox{gray!20} MAINTAINED & '
    for sf in steering_features:
        cellcontent = round(maintained_dim_avg_success[sf],3)
        latex_code += '\\textbf{' + str(maintained_dims[sf]) + f'({cellcontent})' +'} & '
    latex_code = latex_code[:-2] + ' \\\\\n'

    latex_code += 'UNCONTROLLED & '
    for sf in steering_features:
        latex_code += '\\textbf{' + str(uncontroll_dims[sf]) +'} & '

    latex_code = latex_code[:-2] + ' \\\\\n\\bottomrule\n'
    latex_code += '\\end{tabular}\n}\n\\end{center}\n\\end{table*}'
    print(latex_code)
    #write the latex code to a file
    with open(table1_name+'.tex', 'w') as f:
        f.write(latex_code)

get_latex_table1_deprecated(table1_gemma, 'table1_gemma')

#OK, nice job. Now let's make another form of the latex table. This time, the rows will be the steering features and the columns will be the value dimensions. 
#The cells will contain the success rate of steering the value dimension using the steering feature.
#To avoid making the table too wide, the string of value dimensions will be rotated 90 degrees.
#Let's begin
def get_latex_table1_rotate_deprecated(table1, table1_name):
    latex_code = '\\begin{table*}[ht]\n\\caption{Value steering using SAE features for the Gemma-2B-IT model. Expected stimulated values are highlighted in red, along with their actual success rate during testing. Expected suppressed values are marked in Purple. Maintained values are shown in gray. Light red indicates values that are expected to be at least not suppressed, while light purple represents values that are expected to be at least not stimulated. Blank cells correspond to uncontrollable values. The bottom of the table indicates the count of each of the six expected categories and their average success rates.}\n\\label{table: sae-steering-gemma}\n\\begin{center}\n\\scalebox{0.5}{'

    latex_code += '\\begin{tabular}{' + 'c@{\\hspace{1.5pt}}|' * (len(table1.index) + 6) + 'c' + '}\n\\toprule\n'
    #latex_code += '\\begin{tabular}{' + 'c|' * len(table1.index) + 'c' + '}\n\\toprule\n'
    #transfer table1.columns to a list of str

    value_dims = list(map(str, table1.index))
    steering_features = table1.columns
    latex_code += 'Value & ' + ' & '.join(['\\rotatebox{90}{\\bf ' + tc +'}' for tc in value_dims]) + ' &\\rotatebox{90} {\\bf STIMULATE} & \\rotatebox{90} {\\bf SUPPRESS} & \\rotatebox{90} {\\bf NON-SUPPRESS}& \\rotatebox{90} {\\bf NON-STIMULATE} & \\rotatebox{90} {\\bf MAINTAIN} & \\rotatebox{90} {\\bf UNCONTROLLED} \\\\\n\\hline\n'
    


    for sf in steering_features:
        stimulated_dim_avg_success = []
        stimulhalf_dim_avg_success = []
        suppressed_dim_avg_success = []
        supprehalf_dim_avg_success = []
        maintained_dim_avg_success = []


        uncontroll_dims = 0
        stimulated_dims = 0
        suppressed_dims = 0
        stimulhalf_dims = 0
        supprehalf_dims = 0
        maintained_dims = 0

        latex_code += '\\small ' + str(sf) + ' & '
        for vd in value_dims:
            value = table1.loc[vd, sf]
            if type(value) == str:
                value = value.split(',')
                if value[0] == 'STIMULATE':
                    stimulated_dim_avg_success.append(float(value[1]))
                    stimulated_dims += 1
                    latex_code += '\\colorbox{red!50}' + '{' + f"{float(value[1]):.2f}" + '} & '
                elif value[0] == 'NON_SUPPRESS':
                    stimulhalf_dim_avg_success.append(float(value[1]))
                    stimulhalf_dims += 1
                    latex_code += '\\colorbox{red!20}' + '{' + f"{float(value[1]):.2f}" + '} & '
                elif value[0] == 'SUPPRESS':
                    suppressed_dim_avg_success.append(float(value[1]))
                    suppressed_dims += 1
                    latex_code += '\\colorbox{blue!50}' + '{' + f"{float(value[1]):.2f}" + '} & '
                elif value[0] == 'NON_STIMULATE':
                    supprehalf_dim_avg_success.append(float(value[1]))
                    supprehalf_dims += 1
                    latex_code += '\\colorbox{blue!20}' + '{' + f"{float(value[1]):.2f}" + '} & '
                elif value[0] == 'MAINTAIN':
                    maintained_dim_avg_success.append(float(value[1]))
                    maintained_dims += 1
                    latex_code += '\\colorbox{gray!20}' + '{' + f"{float(value[1]):.2f}" + '} & '
                else:
                    raise ValueError('Invalid value')
            else:
                assert np.isnan(value)
                uncontroll_dims += 1
                latex_code += f"-" + ' & '
        latex_code = latex_code[:-2] + ' & ' + str(stimulated_dims) + f'({round(np.mean(stimulated_dim_avg_success),3)})' + ' & ' + str(suppressed_dims) + f'({round(np.mean(suppressed_dim_avg_success),3)})' + ' & ' + str(stimulhalf_dims) + f'({round(np.mean(stimulhalf_dim_avg_success),3)})'  + ' & ' + str(supprehalf_dims) + f'({round(np.mean(supprehalf_dim_avg_success),3)})' + ' & ' + str(maintained_dims) + f'({round(np.mean(maintained_dim_avg_success),3)})' + ' & ' + str(uncontroll_dims) + ' \\\\\n'

        latex_code = latex_code[:-2] + ' \\\\\n'
    latex_code = latex_code + ' \\midrule\n'
    
    latex_code = latex_code + ' \\\\\n\\bottomrule\n'



    latex_code += '\\end{tabular}\n}\n\\end{center}\n\\end{table*}'
    print(latex_code)
    #write the latex code to a file
    with open(table1_name+'.tex', 'w') as f:
        f.write(latex_code)
    
get_latex_table1_rotate_deprecated(table1_gemma, 'table1_gemma_rotate')

In [ ]:
def get_latex_table1_rotate_new_dep(table1, table1_name):
    latex_code = '\\begin{table*}[ht]\n\\caption{Value steering using SAE features for the Gemma-2B-IT model. Expected stimulated values are highlighted in red, along with their actual success rate during testing. Expected suppressed values are marked in Purple. Maintained values are shown in gray. Light red indicates values that are expected to be at least not suppressed, while light purple represents values that are expected to be at least not stimulated. Blank cells correspond to uncontrollable values. The bottom of the table indicates the count of each of the six expected categories and their average success rates.}\n\\label{table: sae-steering-gemma}\n\\begin{center}\n\\scalebox{0.5}{'

    latex_code += '\\begin{tabular}{' + 'c@{\\hspace{1.5pt}}|' * (len(table1.index) + 1) + 'c' + '}\n\\toprule\n'
    #latex_code += '\\begin{tabular}{' + 'c|' * len(table1.index) + 'c' + '}\n\\toprule\n'
    #transfer table1.columns to a list of str

    value_dims = list(map(str, table1.index))
    steering_features = table1.columns
    latex_code += '\\rotatebox{90}{Value} & ' + ' & '.join(['\\rotatebox{90}{\\bf ' + tc +'}' for tc in value_dims]) + '&\\rotatebox{90}{\\bf AVG}' + ' \\\\\n\\hline\n'
    # &\\rotatebox{90} {\\bf STIMULATE} & \\rotatebox{90} {\\bf SUPPRESS} & \\rotatebox{90} {\\bf NON-SUPPRESS}& \\rotatebox{90} {\\bf NON-STIMULATE} & \\rotatebox{90} {\\bf MAINTAIN} & \\rotatebox{90} {\\bf UNCONTROLLED}


    for sf in steering_features:
        cosines = []

        latex_code += '\\small ' + str(int(sf)) + ' & '
        for vd in value_dims:
            value = table1.loc[vd, sf]
            if type(value) == str:
                #split value by / and trans each part to int
                value = value.split('/')
                value = list(map(int, value))
                #value = [red, blue, transparency, red_test, blue_test, transparency_test]
                traindata = value[:3]
                testdata = value[3:]
                #compute the cosine similarity between the two vectors (first normalize them)
                traindata_p = np.array(traindata) / np.sum(traindata)
                testdata_p = np.array(testdata) / np.sum(testdata)

                if vd.startswith('Anxiety') and sf == 1312:
                    pass
                #cosine similarity
                similarity = np.dot(traindata_p, testdata_p) / (np.linalg.norm(traindata_p) * np.linalg.norm(testdata_p))
                cosines.append(similarity)


                coeff = 1
                transratio = value[2] / (value[0] + value[1] + value[2])
                redratio = (value[0] / (value[0] + value[1])) 
                blueratio = (value[1] / (value[0] + value[1]))  
                redratio_t = redratio * (1 - transratio) 
                blueratio_t = blueratio * (1 - transratio)

                rr = int(255 * (1 - blueratio_t) * redratio_t + 255 * (1 - redratio_t) * (1 - blueratio_t)) / 255.0
                gg = int(255 * (1 - redratio_t) * blueratio_t + 255 * (1 - blueratio_t) * (1 - redratio_t)) / 255.0
                bb = int(255 * (1 - redratio_t) * (1 - blueratio_t))  / 255.0

                #latex_code += '\\colorbox{red!' + str(redratio_t) + '!green!' + str(blueratio_t) + '}' + '{' + f"{float(similarity):.2f}" + '} & '
                #latex_code += '\\colorbox[rgb]{' + str(redratio_t) + ',' + str(blueratio_t) + ',0' + '}' + '{' + f"{float(similarity):.2f}" + '} & '
                latex_code += '\\colorbox[rgb]{' + str(round(rr, 2)) + ',' + str(round(gg, 2)) + ',' + str(round(bb, 2)) + '}' + '{' + f"{float(similarity):.2f}" + '} & '
                #latex_code += '\\colorbox[rgb]{' + str(round(bb, 2)) + ',' + str(round(rr, 2)) + ',' + str(round(gg, 2)) + '}' + '{' + f"{float(similarity):.2f}" + '} & '
            

                # if random.random() < 0.5:
                #     latex_code += '\\colorbox[rgb]{1,0,0}' + '{' + f"{float(similarity):.2f}" + '} & '
                # else:
                #     latex_code += '\\colorbox[rgb]{0.14,0.78,0}' + '{' + f"{float(similarity):.2f}" + '} & '

        # latex_code = latex_code[:-2] + ' & ' + str(stimulated_dims) + f'({round(np.mean(stimulated_dim_avg_success),3)})' + ' & ' + str(suppressed_dims) + f'({round(np.mean(suppressed_dim_avg_success),3)})' + ' & ' + str(stimulhalf_dims) + f'({round(np.mean(stimulhalf_dim_avg_success),3)})'  + ' & ' + str(supprehalf_dims) + f'({round(np.mean(supprehalf_dim_avg_success),3)})' + ' & ' + str(maintained_dims) + f'({round(np.mean(maintained_dim_avg_success),3)})' + ' & ' + str(uncontroll_dims) + ' \\\\\n'

        latex_code = latex_code[:-2] + ' & ' + f"{np.mean(cosines):.2f}" + ' \\\\\n'
    #latex_code = latex_code + ' \\midrule\n'
    latex_code = latex_code + ' \\\\\n\\bottomrule\n'



    latex_code += '\\end{tabular}\n}\n\\end{center}\n\\end{table*}'
    #print(latex_code)
    #write the latex code to a file
    with open(table1_name+'.tex', 'w') as f:
        f.write(latex_code)
    
get_latex_table1_rotate_new(table1_gemma, 'table1_gemma_rotate')